## How To Perform a Dispersive Shift Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll be measuring the dispersive shift of the readout resonator.

Measurements on superconducting qubits such as Transmons are often based on dispersive readout. Here, the qubit itself is not directly probed, instead a resonator, which is coupled capacitively to the qubit, is probed with a measure pulse close to its resonance frequency (see figure below). The amplitude and phase of the returning measure pulse (here transmitted) contain information about the qubit state.

Dispersive limit is a regime where the qubit frequency ($\omega_Q$) is far detuned from the resonance frequency of the resonator ($\omega_R$) such that its detuning ($\Delta = |\omega_Q-\omega_R|$) is large compared to the coupling strength ($g \ll \Delta$). In this regime, the cavity frequency experiences a dispersive shift,

$$\pm\chi = \pm\frac{g^2}{\Delta}$$

dependent on the qubit state. This is illustrated in the figure below that shows the transmitted amplitude and phase responses of the readout resonator when qubit is in its ground and excited states. Higher excited stats of the qubits can be similarly measured through dispersive readout.

![Transmitted amplitude and phase response of the readout resonator coupled to a qubit in state g, e and f. When the resonator is probed, the transmitted amplitude and phase contain information about the qubit state.](../images/dispersive_ro_1_gef.svg "Transmitted amplitude and phase response of the readout resonator coupled to a qubit in state g, e and f. When the resonator is probed, the transmitted amplitude and phase contain information about the qubit state.")

In the dispersive limit, there is no direct exchange of energy between the two systems. Hence, we minimize back action on the qubit (quantum non-demolition (QND) measurement).

### Imports

You'll start by importing the qubit spectroscopy experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [1]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import (
    dispersive_shift,
)
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [2]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Qubit Spectroscopy Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

my_workflow = dispersive_shift.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubits[0],
    frequencies=np.linspace(6.8e9, 7.1e9, 101),
    states = "ge",
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_qubit_spec = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_qubit_spec, length=10e-6)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `dispersive_shift` to see how the experiment pulse sequence is created:

In [ ]:
print(dispersive_shift.create_experiment.src)

### Changing the Options

We can give our Qubit Spectroscopy experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = dispersive_shift.experiment_workflow.options()
my_new_opts

Then provide new options. Let's change the counts

In [8]:
my_new_opts.count(2048)

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. Here, we change the states into "gef" to map the resonator dispersive shift at both e and f qubit states. You'll notice that the first part of the sequence does not have a drive pulse for g state measurement, followed by a single ge transition pulse for e state measurement, and lastly contain both ge and ef transition pulses for f state measurement.

In [ ]:
my_new_workflow = dispersive_shift.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubits[0],
    frequencies=np.linspace(6.8e9, 7.1e9, 101),
    states = "gef",
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_qubit_spec = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_qubit_spec, length=10e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the qubit spectroscopy experiment with different settings by passing it a copy of the qubits with modified parameters:

Let's reduce the readout length and reset delay length to better see the qubit drive pulses.

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the reset delay length of the drive pulses
temp_qubits[0].parameters.reset_delay_length = 0.1e-6
temp_qubits[0].parameters.readout_length = 0.5e-6
temp_qubits[0].parameters.readout_integration_length = 0.5e-6


my_new_workflow = dispersive_shift.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=temp_qubits[0],
    frequencies=np.linspace(6.8e9, 7.1e9, 101),
    states = "gef",
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_qubit_spec = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_qubit_spec, length=2e-6)

# revert the reset delay length of the drive pulses back to its default value of 1us
temp_qubits[0].parameters.reset_delay_length = 1e-6
temp_qubits[0].parameters.readout_length = 2e-6
temp_qubits[0].parameters.readout_integration_length = 2e-6

Great! You've now run your Dispersive Shift experiment. Check out these other experiments to keep characterizing your qubits:

In [11]:
# TODO: Add experiment links
# TODO: Add Analysis